In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import preprocessing

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# Домашнее задание № 1

Вам необходимо воспользоваться наработками из предыдущих частей задания (подготовка данных, сложные признаки и стратегия валидации) для предсказания оценки (```G3```) на тестовом наборе данных.

# Часть 5. Обучение алгоритма

Целевая переменная принимает дискретные значения [0; 20], поэтому вы можете поставить как задачу классификации, так и задачу регрессии.

!Не забудьте про оптимизацию параметров алгоритмов, у вас же уже есть данные для валидации.

In [2]:
data = pd.read_csv("train_preprocesed.csv")

In [28]:
X = data.drop("G3", axis=1)
y = data["G3"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
def fit_predict_CV(X_train, X_test, y_train, y_test, model_type, params_grid=dict()):
    model = model_type()
    
    model_cv = GridSearchCV(model, params_grid, cv=KFold(), refit=True)#GridSearchCV
    model_cv.fit(X_train, y_train) 

    best_model = model_cv.best_estimator_
    prediction = best_model.predict(X_test)
    score = accuracy_score(y_test, prediction)
    
    print("Best hyperparameters: ", model_cv.best_params_)
    print("Test accuracy:        ", round(score, 7))
    
    return best_model

## KNN

In [5]:
grid = {"n_neighbors": np.arange(1, 15)}
knn = fit_predict_CV(X_train, X_test, y_train, y_test, KNeighborsClassifier, grid)

Best hyperparameters:  {'n_neighbors': 9}
Test accuracy:         0.1098901


## Bayes

In [6]:
bayes = fit_predict_CV(X_train, X_test, y_train, y_test, GaussianNB)

Best hyperparameters:  {}
Test accuracy:         0.043956


## Регрессия

In [7]:
grid = {
    "solver": ['saga', 'lgbfs'],
    "C": [1, 2, 3],
    "penalty": ["l1","l2"],
    "max_iter": [100, 200]
}
regression = fit_predict_CV(X_train, X_test, y_train, y_test, LogisticRegression, grid)

c:\Max\Anaconda\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Max\Anaconda\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Max\Anaconda\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Max\Anaconda\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Max\Anaconda\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Max\Anaconda\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  w

Best hyperparameters:  {'C': 2, 'max_iter': 100, 'penalty': 'l1', 'solver': 'saga'}
Test accuracy:         0.1208791


c:\Max\Anaconda\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


## SVM

In [8]:
grid = {
    "class_weight": ["balanced"],
    "kernel": ["linear", "poly", "rbf"],
    "degree": np.arange(1, 10)
}
svm = fit_predict_CV(X_train, X_test, y_train, y_test,SVC, grid)

Best hyperparameters:  {'class_weight': 'balanced', 'degree': 1, 'kernel': 'linear'}
Test accuracy:         0.1648352


## Постройте сложную модель на основе предыдущих четырех простых и оцените ее точность

Вариант 1. Нужно взвесить предсказания простых моделей. Предложите метод взвешивания.

In [9]:
def my_prediction(X):
    """ 
    Предсказываемый класс - класс, в котором уверено большинство моделей. 
    Если все модели выдали различный результат, то отдаём предпочтение SVM.
    """
    models_preds = np.array([[svm.predict(X)], [knn.predict(X)], [bayes.predict(X)], [regression.predict(X)]]).T.reshape(91, 4)
    my_preds = []

    for i in range(models_preds.shape[0]):
        my_preds += [np.argmax(np.bincount(models_preds[i]))]

    return my_preds

In [10]:
my_pred = my_prediction(X_test)
score = accuracy_score(y_test, my_pred)
print("Monster accuracy:", round(score, 3))

Monster accuracy: 0.11


Вариант 2. Нужно оценить корреляцию предсказаний простых моделей.

In [11]:
prediction_data = dict()
prediction_data["knn"] = knn.predict(X_train)
prediction_data["bayes"] = bayes.predict(X_train)
prediction_data["reg"] = regression.predict(X_train)
prediction_data["svm"] = svm.predict(X_train)
df_prediction = pd.DataFrame(prediction_data)

In [12]:
df_prediction.corr()

,knn,bayes,reg,svm
knn,1.000000,0.250715,0.464101,0.427966
bayes,0.250715,1.000000,0.442676,0.543952
reg,0.464101,0.442676,1.000000,0.686840
svm,0.427966,0.543952,0.686840,1.000000


## Сделайте итоговое предсказание

Решением домашнего задания является данный ноутбук + .csv файл с предсказанием. Пример формата файла, который должен у вас получится приведен в ```sample_submission.csv```. Данные для предсказания - ```X_test.csv```.

За лучшее предсказание 5 человек получат +10% к домашке.

Значения предсказаний должны быть целочисленными. Проверка будет точности решения будет по MSE.

In [13]:
test_data = pd.read_csv("X_test.csv")

test_data.loc[test_data["sex"] == 'F', "sex"] = 0
test_data.loc[test_data["sex"] == 'M', "sex"] = 1

test_data.loc[test_data["address"] == 'U', "address"] = 0
test_data.loc[test_data["address"] == 'R', "address"] = 1

test_data["sex"] = pd.to_numeric(test_data["sex"])
test_data["address"] = pd.to_numeric(test_data["address"])

cat_columns = []
for name in test_data.columns:
    if name not in test_data._get_numeric_data().columns:
        cat_columns += [name]
test_data = pd.get_dummies(test_data, columns=cat_columns)

In [14]:
best_model = svm

In [15]:
# Для jupyter notebook
submission = pd.read_csv('sample_submission.csv', index_col='id')
submission['G3'] = best_model.predict(test_data)
submission.to_csv('submission.csv', index=False) 

# Часть 6. Состязательная валидация (Adversarial Validation)

Данная стратегия часто применяется в соревнованиях по машинному обучению. Так как проверить свой алгоритм возможно только на заданном тренировочном наборе данных, а результат требуется к набору, для которого нет ответов, то выбирают следующий подход:
- оценить степень схожести тренировочных и тестовых данных;
- составить валидацию из тех тренировочных данных, которые больше похожи на тестовые.

Алгоритм реализации AV:
1. Удаляем ответы из тренировочных данных.
2. Добавляем специальные метки (```is_test```) для классов: тренировочные данные и тестовые.
То есть у тестовых данных будет значение ```is_test = 1```, а у тренировочных ```is_test = 0```.
4. Учим алгоритм классификации предсказывать заданную метку ```is_test```.
5. Делим тренировочные данные на N частей и проводим кросс-валидацию.

Обучаем N алгоритмов классификации: каждый обучается на всех тестовых данных и на N-1 наборах тренировочных. Предсказываем на 1 тренировочном наборе.

6. Для каждого элемента в выборке предсказываем вероятность принадлежности класса и сортируем тренировочные данные по вероятности принадлежности тестовой выборке.
7. Выделяем заданное число данных в валидацию.
8. Обучаемся для итогового предсказания на всех данных.

In [16]:
train_adv = data.drop("G3", axis=1)
test_adv = test_data.copy()

train_adv["is_test"] = 0
test_adv["is_test"] = 1

In [17]:
train_adv.sample(5)

,sex,age,address,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,...,activities_yes,nursery_no,nursery_yes,higher_no,higher_yes,internet_no,internet_yes,romantic_no,romantic_yes,is_test
139,1.0,18.0,1.0,1,2,3,1,0,4,3,...,1,0,1,1,0,0,1,0,1,0
305,1.0,18.0,1.0,3,2,2,1,0,2,5,...,0,1,0,0,1,0,1,1,0,0
89,0.0,18.0,1.0,2,2,3,2,1,4,3,...,1,0,1,0,1,1,0,0,1,0
351,1.0,16.0,0.0,1,2,1,1,0,3,3,...,1,0,1,0,1,0,1,0,1,0
330,0.0,15.0,0.0,2,3,2,1,0,3,5,...,1,0,1,0,1,1,0,1,0,0


In [18]:
test_adv.sample(5)

,sex,age,address,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,...,activities_yes,nursery_no,nursery_yes,higher_no,higher_yes,internet_no,internet_yes,romantic_no,romantic_yes,is_test
27,0,17,0,2,2,1,4,0,3,4,...,1,0,1,0,1,0,1,0,1,1
75,0,15,1,2,2,1,1,0,4,3,...,1,0,1,0,1,1,0,1,0,1
35,0,17,0,2,4,1,2,0,4,3,...,1,0,1,0,1,0,1,0,1,1
99,0,16,0,3,3,2,1,0,4,3,...,1,1,0,0,1,0,1,0,1,1
177,1,17,0,1,1,3,2,0,5,1,...,0,0,1,0,1,0,1,0,1,1


In [19]:
data_adv = pd.concat([train_adv, test_adv])
X_train = data_adv.drop("is_test", axis=1)
y_train = data_adv["is_test"]

In [20]:
grid = {
    "class_weight": ["balanced"],
    "kernel": ["linear", "poly", "rbf"],
    "degree": np.arange(1, 10)
}
# Нет у меня тестовых данных для этой задачи, поэтому вместо теста
# ещё раз прогоняю модель на трейне (Это костыль, признаю)
svm_adv = fit_predict_CV(X_train, X_train, y_train, y_train, SVC, grid)

Best hyperparameters:  {'class_weight': 'balanced', 'degree': 1, 'kernel': 'linear'}
Test accuracy:         0.6317411


In [21]:
mask = svm_adv.predict(train_adv.drop("is_test", axis=1))

In [22]:
bool_mask = np.array(mask, dtype=np.bool8)
bool_mask = np.array([not el for el in bool_mask])
bool_mask_array = np.array([bool_mask]*55).T
val_data = data.mask(bool_mask_array).dropna()

In [23]:
score = accuracy_score(val_data["G3"], svm.predict(val_data.drop("G3", axis=1)))
print("Val accuracy: ", round(score, 7))

Val accuracy:  0.61875


Прикольный результат. Наверное стоит слегка переобучить модель именно на этих данных, чтоб получить больший score. Делать я этого, конечто же, не буду.